# Validação: BERTimbau Large

**BERTimbau Large - Maior capacidade que base**

## 📊 Histórico
- BERTimbau BASE + Focal = 0.79696
- BERTimbau v4 (threshold tuning) = 0.82073
- **Large NUNCA testado sistematicamente**

## 🎯 Objetivo
Testar se o modelo LARGE supera o base.

## 📊 Configurações Testadas
1. Large vs Base (comparação direta)
2. Focal Loss γ=1,2,3
3. Threshold Tuning

---

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from transformers import (
    AutoModelForSequenceClassification, AutoTokenizer,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if os.path.exists('/kaggle/input'):
    DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
    # Busca BERTimbau Large
    def find_model_path():
        base = '/kaggle/input'
        def search_dir(directory, depth=0, max_depth=10):
            if depth > max_depth: return None
            try:
                for item in os.listdir(directory):
                    path = os.path.join(directory, item)
                    if os.path.isdir(path) and os.path.exists(os.path.join(path, 'config.json')):
                        return path
                    result = search_dir(path, depth + 1, max_depth) if os.path.isdir(path) else None
                    if result: return result
            except: pass
            return None
        return search_dir(base)
    MODEL_PATH = find_model_path()
else:
    DATA_DIR = '../data'
    MODEL_PATH = 'neuralmind/bert-large-portuguese-cased'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
print(f'Model: {MODEL_PATH}')

In [ ]:
# ===== DADOS =====
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['report'].tolist(),
    train_df['target'].tolist(),
    test_size=0.2,
    stratify=train_df['target'],
    random_state=SEED
)

print(f'Train: {len(train_texts)}, Val: {len(val_texts)}')

In [ ]:
# ===== DATASET & FOCAL LOSS =====
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        enc = self.tokenizer(str(self.texts[idx]), truncation=True,
                             max_length=self.max_length, padding='max_length', return_tensors='pt')
        return {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        
    def forward(self, inputs, targets):
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        return (self.alpha * (1 - pt) ** self.gamma * ce_loss).mean()

class FocalLossTrainer(Trainer):
    def __init__(self, focal_gamma=2.0, focal_alpha=0.25, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_loss = FocalLoss(gamma=focal_gamma, alpha=focal_alpha)
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop('labels')
        outputs = model(**inputs)
        loss = self.focal_loss(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
# ===== FUNÇÃO DE TREINO =====
def train_and_evaluate(config, tokenizer):
    print(f"\n{'='*60}")
    print(f"Config: {config['name']}")
    print(f"{'='*60}")
    
    train_ds = TextDataset(train_texts, train_labels, tokenizer, config['max_length'])
    val_ds = TextDataset(val_texts, val_labels, tokenizer, config['max_length'])
    
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_PATH, num_labels=7, local_files_only=True
    )
    
    def compute_metrics(eval_pred):
        preds = np.argmax(eval_pred.predictions, axis=1)
        return {'f1_macro': f1_score(eval_pred.label_ids, preds, average='macro')}
    
    args = TrainingArguments(
        output_dir='/tmp/bertimbau_large_val',
        num_train_epochs=config.get('epochs', 5),
        per_device_train_batch_size=config['batch_size'],
        per_device_eval_batch_size=config['batch_size']*2,
        learning_rate=config['lr'],
        weight_decay=0.01,
        warmup_ratio=0.1,
        fp16=True,
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        report_to='none',
        seed=SEED,
        gradient_accumulation_steps=config.get('grad_accum', 1),
    )
    
    if config.get('focal_gamma'):
        trainer = FocalLossTrainer(
            focal_gamma=config['focal_gamma'],
            model=model, args=args,
            train_dataset=train_ds, eval_dataset=val_ds,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
        )
    else:
        trainer = Trainer(
            model=model, args=args,
            train_dataset=train_ds, eval_dataset=val_ds,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
        )
    
    import time
    start = time.time()
    trainer.train()
    train_time = time.time() - start
    
    results = trainer.evaluate()
    results['train_time'] = train_time
    
    preds = trainer.predict(val_ds)
    y_pred = np.argmax(preds.predictions, axis=1)
    
    print(f"F1-Macro: {results['eval_f1_macro']:.5f}, Tempo: {train_time/60:.1f} min")
    print(classification_report(val_labels, y_pred))
    
    del model, trainer
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return results, y_pred, preds.predictions

In [ ]:
# ===== TOKENIZER =====
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
print('Tokenizer carregado!')

In [ ]:
# ===== EXPERIMENTO 1: CrossEntropy =====
# LARGE precisa de batch menor (mais parâmetros)
config_ce = {'name': 'Large_CE', 'lr': 2e-5, 'batch_size': 4, 'max_length': 256, 'grad_accum': 2}
results_ce, preds_ce, logits_ce = train_and_evaluate(config_ce, tokenizer)

In [ ]:
# ===== EXPERIMENTO 2: Focal γ=2 =====
config_focal2 = {'name': 'Large_Focal_g2', 'lr': 2e-5, 'batch_size': 4, 'max_length': 256, 
                  'grad_accum': 2, 'focal_gamma': 2.0}
results_focal2, preds_focal2, logits_focal2 = train_and_evaluate(config_focal2, tokenizer)

In [ ]:
# ===== EXPERIMENTO 3: Focal γ=3 =====
config_focal3 = {'name': 'Large_Focal_g3', 'lr': 2e-5, 'batch_size': 4, 'max_length': 256,
                  'grad_accum': 2, 'focal_gamma': 3.0}
results_focal3, preds_focal3, logits_focal3 = train_and_evaluate(config_focal3, tokenizer)

In [ ]:
# ===== EXPERIMENTO 4: LR mais baixo =====
config_lr = {'name': 'Large_Focal_lr1e5', 'lr': 1e-5, 'batch_size': 4, 'max_length': 256,
              'grad_accum': 2, 'focal_gamma': 2.0}
results_lr, preds_lr, logits_lr = train_and_evaluate(config_lr, tokenizer)

In [ ]:
# ===== THRESHOLD TUNING =====
from scipy.special import softmax

def find_best_thresholds(logits, labels):
    probs = softmax(logits, axis=1)
    best_thresholds = {}
    
    for c in range(7):
        best_f1, best_t = 0, 0.5
        for t in np.arange(0.1, 0.9, 0.05):
            preds = np.argmax(probs, axis=1)
            mask = probs[:, c] > t
            preds_adj = preds.copy()
            preds_adj[mask] = c
            f1 = f1_score(labels, preds_adj, average='macro')
            if f1 > best_f1:
                best_f1, best_t = f1, t
        best_thresholds[c] = best_t
    return best_thresholds

# Usar melhor modelo
best_logits = logits_focal2
thresholds = find_best_thresholds(best_logits, val_labels)

print("Thresholds:")
for c, t in thresholds.items():
    print(f"  Classe {c}: {t:.2f}")

# Aplicar
probs = softmax(best_logits, axis=1)
preds_t = np.argmax(probs, axis=1)
for c, t in thresholds.items():
    preds_t[probs[:, c] > t] = c

f1_t = f1_score(val_labels, preds_t, average='macro')
print(f"\nF1 com Thresholds: {f1_t:.5f}")

In [ ]:
# ===== RESUMO =====
print("\n" + "="*60)
print("📊 RESUMO - BERTimbau Large Validation")
print("="*60)

all_results = [
    ('Large_CE', results_ce['eval_f1_macro']),
    ('Large_Focal_g2', results_focal2['eval_f1_macro']),
    ('Large_Focal_g3', results_focal3['eval_f1_macro']),
    ('Large_Focal_lr1e5', results_lr['eval_f1_macro']),
    ('+ Thresholds', f1_t),
]

for name, f1 in sorted(all_results, key=lambda x: -x[1]):
    print(f"{name:<20} {f1:.5f}")

print(f"\n📝 Base + Focal: 0.79696")
print(f"📝 Base v4 (threshold): 0.82073")

In [ ]:
# ===== INSIGHTS =====
print("""
📝 INSIGHTS - BERTimbau Large
============================

1. **Large vs Base:**
   - [PREENCHER APÓS EXPERIMENTOS]
   - Large tem 335M vs 110M parâmetros
   - Precisa de batch menor e mais memória

2. **Focal Loss:**
   - [PREENCHER]

3. **Threshold Tuning:**
   - [PREENCHER]

4. **Recomendação:**
   - Se Large > Base: usar Large para competição
   - Se Base ≈ Large: manter Base (mais rápido)
""")